In [11]:
import numpy as np
import pandas as pd
from sklearn import metrics
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [3]:
pos_df = pd.read_csv(r'C:\Users\Roydon\Desktop\it1244-final-project\data\pos.csv', index_col=0)
pos_df = pos_df.drop(columns=['FileName'])
pos_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Content  25000 non-null  object
 1   rating   25000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 585.9+ KB


In [4]:
neg_df = pd.read_csv(r'C:\Users\Roydon\Desktop\it1244-final-project\data\neg.csv', index_col=0)
neg_df = neg_df.drop(columns=['FileName'])
neg_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Content  25000 non-null  object
 1   rating   25000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 585.9+ KB


In [5]:
# Bring all positive rating values down to range of 5-8
pos_df['rating'] = pos_df['rating'] - 2
pos_df['rating'].describe()

count    25000.00000
mean         6.77064
std          1.15738
min          5.00000
25%          6.00000
50%          7.00000
75%          8.00000
max          8.00000
Name: rating, dtype: float64

In [6]:
# Merging data into one dataset
data = pd.concat([pos_df, neg_df], ignore_index=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Content  50000 non-null  object
 1   rating   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [13]:
train_size = 0.8
train_dataset=data.sample(frac=train_size,random_state=200)
test_dataset=data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

FULL Dataset: (50000, 2)
TRAIN Dataset: (40000, 2)
TEST Dataset: (10000, 2)


In [14]:
train_dataset.to_csv('processed_ratings_train.csv')
test_dataset.to_csv('processed_ratings_test.csv')

In [12]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 200
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

c:\Users\Roydon\anaconda3\envs\hpc\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Roydon\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
class CustomDataset(Dataset):
    '''This class accepts `tokenizer`, `dataframe` and `max_length` as input and generates tokenized output and tags that is used by the BERT model for training. 
    The BERT tokenizer is used for tokenizing the data in the `Content` column of the dataframe.
    The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs: `ids`, `attention_mask`, `token_type_ids`'''

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.Content = dataframe.Content
        self.targets = self.data.list
        self.max_len = max_len

    def __len__(self):
        return len(self.Content)

    def __getitem__(self, index):
        Content = str(self.Content[index])
        Content = " ".join(Content.split())

        inputs = self.tokenizer.encode_plus(
            Content,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }